In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from scipy.optimize import minimize
from tqdm import tqdm
import sqlite3
import pickle
import matplotlib.pyplot as plt

# Re-extract the SQL data and perform the necessary data cleaning
# Suppose the Maotai stocks closing price data is saved in a DataFrame named "maotai", which columns called "Date" and "Close", and the process of data cleansing has been finished.

# Get data from database and import it into a csv file

database_path = "C:/Users/fortu/mydatabase.sqlite"

# connect to database
conn = sqlite3.connect(database_path)
cursor = conn.cursor()

# Fetch data from table
query_maotai = "SELECT * FROM MAOTAI"
maotai = pd.read_sql_query(query_maotai, conn)

query_hs300 = "SELECT * FROM HS300"
hs300 = pd.read_sql_query(query_hs300, conn)

query_bond10 = "SELECT * FROM BOND10"
bond10 = pd.read_sql_query(query_bond10, conn)

query_credit = "SELECT * FROM CREDIT"
credit = pd.read_sql_query(query_credit, conn)

query_zz500 = "SELECT * FROM ZZ500"
zz500 = pd.read_sql_query(query_zz500, conn)

query_bond3_5 = "SELECT * FROM 'BOND3-5'"
bond3_5 = pd.read_sql_query(query_bond3_5, conn)

# close the connection with database
cursor.close()
conn.close()

# Delete the line containing missing values
maotai.dropna(inplace=True)
hs300.dropna(inplace=True)
bond3_5.dropna(inplace=True)
bond10.dropna(inplace=True)
zz500.dropna(inplace=True)
credit.dropna(inplace=True)

# Rename the column of DataFrame
maotai.rename(columns={"mt_close": "Close"}, inplace=True)
hs300.rename(columns={"hs300": "Close"}, inplace=True)
bond10.rename(columns={"bond10": "Close"}, inplace=True)
bond3_5.rename(columns={"bond3-5": "Close"}, inplace=True)
credit.rename(columns={"credit": "Close"}, inplace=True)
zz500.rename(columns={"zz500": "Close"}, inplace=True)

# Replace missing values with Nan, as well as convert STRING type to FLOAT type
maotai["Close"] = maotai["Close"].replace('', float('nan'))
maotai["Close"] = maotai["Close"].astype(float)

hs300["Close"] = hs300["Close"].replace('', float('nan'))
hs300["Close"] = hs300["Close"].astype(float)

bond10["Close"] = bond10["Close"].replace('', float('nan'))
bond10["Close"] = bond10["Close"].astype(float)

bond3_5["Close"] = bond3_5["Close"].replace('', float('nan'))
bond3_5["Close"] = bond3_5["Close"].astype(float)

credit["Close"] = credit["Close"].replace('', float('nan'))
credit["Close"] = credit["Close"].astype(float)

zz500["Close"] = zz500["Close"].replace('', float('nan'))
zz500["Close"] = zz500["Close"].astype(float)

rm = hs300["Close"]
rf = 0.02

maotai["Date"] = pd.to_datetime(maotai["Date"])

hs300['Date'] = pd.to_datetime(hs300['Date'])

zz500['Date'] = pd.to_datetime(zz500['Date'])

bond10['Date'] = pd.to_datetime(bond10['Date'])

bond3_5['Date'] = pd.to_datetime(bond3_5['Date'])

credit['Date'] = pd.to_datetime(credit['Date'])

# maotai.plot(x="Date", y="Close", title="Maotai Stock Close Price")
# hs300.plot(x="Date", y="Close", title="HS300 Index Close Price")
# # plt.show()

maotai["Close"].fillna(0, inplace=True)
credit["Close"].fillna(0, inplace=True)
bond3_5["Close"].fillna(0, inplace=True)
bond10["Close"].fillna(0, inplace=True)
zz500["Close"].fillna(0, inplace=True)
hs300["Close"].fillna(0, inplace=True)

maotai["Close"].interpolate(method='linear', inplace=True)
hs300["Close"].interpolate(method='linear', inplace=True)

std = maotai["Close"].std()
mean = maotai["Close"].mean()
maotai["Close"] = np.where(maotai["Close"] > mean + 3 * std, mean + 3 * std, maotai["Close"])
maotai["Close"] = np.where(maotai["Close"] < mean - 3 * std, mean - 3 * std, maotai["Close"])

std_hs300 = hs300["Close"].std()
mean_hs300 = hs300["Close"].mean()
hs300["Close"] = np.where(hs300["Close"] > mean_hs300 + 3 * std_hs300, mean_hs300 + 3 * std_hs300, hs300["Close"])
hs300["Close"] = np.where(hs300["Close"] < mean_hs300 - 3 * std_hs300, mean_hs300 - 3 * std_hs300, hs300["Close"])

maotai["z-score"] = maotai["Close"].rolling(window=60).apply(lambda x: (x[-1] - x.mean()) / x.std(), raw=True)
hs300["z-score"] = hs300["Close"].rolling(window=60).apply(lambda x: (x[-1] - x.mean()) / x.std(), raw=True)

maotai.set_index("Date", inplace=True)
maotai_monthly = maotai.resample("M").last()
maotai_monthly["Momentum"] = (
        maotai_monthly["Close"].pct_change(periods=12) - maotai_monthly["Close"].pct_change(periods=1)).dropna()

hs300.set_index("Date", inplace=True)
hs300_monthly = hs300.resample("M").last()
hs300_monthly["Momentum"] = (hs300_monthly["Close"].pct_change(periods=12) - hs300_monthly["Close"].pct_change(periods=1)).dropna()

maotai_yearly = maotai.resample("Y").last()
maotai_yearly["Yearly Return"] = maotai_yearly["Close"].pct_change()

hs300_yearly = hs300.resample("Y").last()
hs300_yearly["Yearly Return"] = hs300_yearly["Close"].pct_change()

# Build the DataFrame of the CAPM model
data = pd.DataFrame({'Rm': hs300_yearly["Yearly Return"], 'Rf': 0.02})  # 使用沪深300指数作为市场风险溢价，假设无风险利率为2%
data['Ri'] = maotai_yearly["Yearly Return"] - data['Rf']  # 计算茅台股票的超额收益率
data.dropna(inplace=True)

# Add constant terms
X = sm.add_constant(data['Rm'])

# Fit the CAPM model
model = sm.OLS(data['Ri'], X)
results = model.fit()

# Result
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                     Ri   R-squared:                       0.513
Model:                            OLS   Adj. R-squared:                  0.452
Method:                 Least Squares   F-statistic:                     8.425
Date:                Thu, 13 Apr 2023   Prob (F-statistic):             0.0198
Time:                        14:12:40   Log-Likelihood:                -2.2259
No. Observations:                  10   AIC:                             8.452
Df Residuals:                       8   BIC:                             9.057
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.2439      0.113      2.158      0.0

C:\Users\fortu\AppData\Roaming\Python\Python310\site-packages\scipy\stats\_stats_py.py:1772: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=10
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "
